In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!nvidia-smi

Thu Oct 27 10:39:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import datetime
import time
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

import pathlib
import seaborn as sns

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub

In [ ]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please turn on Colab GPU Runtime")

Default GPU Device: /device:GPU:0


In [ ]:
data_dir = pathlib.Path('/content/gdrive/MyDrive/CS4243/data')

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3960


In [ ]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000, seed=42):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=seed)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
list_ds = list_ds.shuffle(image_count, seed=42, reshuffle_each_iteration=False)

In [ ]:
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

['carrying' 'normal' 'threat']


In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(list_ds, image_count, shuffle=False)

In [ ]:
def get_label(file_path):
  # Convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

def decode_img(img, image_size):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return tf.image.resize(img, image_size)

def parse_image(file_path, image_size):
  label = get_label(file_path)
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img, image_size)
  return img, label

In [ ]:
image_size = [224, 224]
batch_size = 32
buffer_size = 100

In [ ]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds = train_ds.map(lambda x: parse_image(x, image_size), num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(lambda x: parse_image(x, image_size), num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(lambda x: parse_image(x, image_size), num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
normalize = layers.Rescaling(1./255)
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, batch_size=32, shuffle=False, augment=False):
  # Resize and rescale all datasets.
  ds = ds.map(lambda x, y: (normalize(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets.
  ds = ds.batch(batch_size)

  # Use data augmentation only on the training set.
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

In [ ]:
class LeNet(keras.Model):
  def __init__(self, num_classes=10):
    super(LeNet, self).__init__()
    self.conv1 = layers.Conv2D(6, 5, padding='same', activation='relu')
    self.pool1 = layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2))
    self.conv2 = layers.Conv2D(16, 5, padding='same', activation='relu')
    self.pool2 = layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2))
    self.flatten = layers.Flatten()
    self.d1 = layers.Dense(128, activation='relu')
    self.d2 = layers.Dense(84, activation='relu')
    self.final = layers.Dense(num_classes)

  def call(self, x):
    x = self.conv1(x)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d2(x)
    return self.final(x)

# Create an instance of the model
model = LeNet(num_classes=3)

In [ ]:
features, labels = next(iter(train_ds))

print(features.shape)
print(labels.shape)

(32, 224, 224, 3)
(32,)


In [ ]:
predictions = model(features)
predictions[:5]

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[ 5.3305387e-02,  1.4020781e-01, -1.8306246e-01],
       [-9.9119730e-05,  3.0281678e-02, -8.5735612e-02],
       [ 1.7102305e-02,  1.0672427e-01, -1.4694086e-01],
       [ 4.6684887e-02,  7.5394258e-02, -1.3886884e-01],
       [-1.5879069e-02,  1.5665153e-03, -4.2538531e-02]], dtype=float32)>

In [ ]:
predictions = model(features)
predictions[:5]

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[ 5.3305387e-02,  1.4020781e-01, -1.8306246e-01],
       [-9.9119730e-05,  3.0281678e-02, -8.5735612e-02],
       [ 1.7102305e-02,  1.0672427e-01, -1.4694086e-01],
       [ 4.6684887e-02,  7.5394258e-02, -1.3886884e-01],
       [-1.5879069e-02,  1.5665153e-03, -4.2538531e-02]], dtype=float32)>

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

train_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [ ]:
@tf.function
def train_step(x, y, loss_fn, optimizer, train_metric):
    with tf.GradientTape() as tape:
        # training=True is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        preds = model(x, training=True)
        loss_value = loss_fn(y, preds)

    grads = tape.gradient(loss_value, model.trainable_variables)

    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_metric.update_state(y, preds)

    return loss_value

In [ ]:
@tf.function
def val_step(x, y, loss_fn, val_metric):
    val_preds = model(x, training=False)
    val_loss = loss_fn(y, val_preds)
    val_metric.update_state(y, val_preds)
    return val_loss

In [ ]:
def train(model, train_ds, val_ds, epochs, loss_fn, optimizer, train_metric, val_metric):
    ## Note: Rerunning this cell uses the same model parameters

    # Keep results for plotting
    train_acc_per_epoch = []
    val_acc_per_epoch = []
    train_mean_loss = [] # Mean of each epoch
    val_mean_loss = [] # Mean of each epoch

    for epoch in range(epochs):
        train_losses = []
        val_losses = []

        start = time.time()
        step = 0
        # Training loop - using batches of 32
        for x, y in train_ds:
            # Optimize the model
            loss = train_step(x, y, loss_fn, optimizer, train_metric)
            train_losses.append(loss.numpy())

            if step % 50 == 0:
                print(f"Training loss at epoch {epoch + 1}, step {step:d}: {float(loss):.5f}")
            
            step += 1

        for x, y in val_ds:
            # training=False is needed only if there are layers with different
            # behavior during training versus inference (e.g. Dropout).
            loss = val_step(x, y)
            val_losses.append(loss.numpy())
        
        # End epoch
        train_acc = train_metric.result()
        val_acc = val_metric.result()

        train_mean_loss.append(np.mean(train_losses))
        val_mean_loss.append(np.mean(val_losses))

        train_acc_per_epoch.append(train_acc.numpy())
        val_acc_per_epoch.append(val_acc.numpy())

        train_metric.reset_states()
        val_metric.reset_states()

        print(f"Epoch {epoch + 1}/{epochs}: train_loss: {np.mean(train_losses):.4f}, train_acc: {train_acc:.4f}, val_loss: {np.mean(val_losses):.4f}, val_acc: {val_acc:.4f}")

        print(f"Time taken for this epoch: {time.time() - start:.2f}s")
        print('--' * 30)
    history = (train_mean_loss, train_acc_per_epoch, val_mean_loss, val_acc_per_epoch)
    return history

In [ ]:
history = train(model, train_ds, val_ds, 1, loss_fn, optimizer, train_metric, val_metric)

Training loss at epoch 0, step 0: 1.13482
Training loss at epoch 0, step 50: 1.06919


KeyboardInterrupt: ignored